In [68]:
from datetime import datetime, date
import holidays
from holidays.countries import US
import pandas as pd

In [82]:
class NJCourtsHolidays(US):
    """NJ Courts custom holiday calendar.

    Extends UnitedStates holidays with court-specific recess days:
          - Day after Thanksgiving
          - Day after Christmas (only if Dec 25 is Thursday)

    Attributes:
        subdiv (str): Subdivision code ("NJ") by default.
        year (int): Inputted date's year

    Example:
        >>>nj_holidays = NJCourtsHolidays()
        >>>datetime.date(2025, 12, 25) in nj_holidays
        True
        >>>datetime.date(2025, 12, 26) in nj_holidays
        True
        >>>datetime.date(2025, 12, 27) in nj_holidays
        False
        >>>datetime.date(2031, 12, 26) in nj_holidays
        True

    2025 Judiciary Calendar Holidays:
    Jan 1, 20 : New Year’s Day, MLK Day
    Feb 17: President’s Day
    Apr 18: Good Friday
    May 26: Memorial Day
    Jun 20: Juneteenth
    July 4: Independence Day
    Sept 1: Labor Day
    Oct 13: Columbus Day
    Nov 4, 11, 27: Election Day, Veteran’s Day, Thanksgiving
    Dec 25: Christmas Day
    """
    def __init__(self, **kwargs):
        kwargs.setdefault("subdiv", "NJ")
        super().__init__(**kwargs)

    def _populate(self, year):
        super()._populate(year)

        # Remove Lincoln's Birthday 2025-02-12
        self.pop_named("Lincoln's Birthday")

        # Add Election Day - First Tuesday in November
        self._add_holiday_1st_tue_of_nov("Election Day")
        
        # Day after Thanksgiving (Friday)
        self._add_holiday_1_day_past_4th_thu_of_nov("Court Recess (Day after Thanksgiving)")

        # December 26, but ONLY if Dec 25 is a Thursday
        christmas = date(year, 12, 25)
        if christmas.weekday() == 3:  # Thursday = 3
            self._add_holiday_dec_26("Court Recess (Day after Christmas)")

def HolidayChecker(NJCourtsHolidays_Class, date_string):
    """
    Checks if the provided date is a Holiday defined by NJ Courts

    Args:
        NJCourtsHolidays_Class (class): Instance of a NJCourtsHolidays class
        check_date (datetime.date): A string given in Date Format: "MM/DD/YYYY"

    Returns:
        bool: True if the date is a holiday in NJ Courts, False otherwise.
    Example:
        >>>nj_holidays = NJCourtsHolidays()
        >>>date_string = df['transaction_date'].iloc[0] #'4/30/2025'
        >>>HolidayChecker(nj_holidays, date_string)
        False
    """
    check_date = datetime.strptime(date_string, "%m/%d/%Y").date() 
    return check_date in NJCourtsHolidays_Class

def generate_holidays(starting_year=2025, ending_year=2036):
    """
    Generate a DataFrame of holidays defined by `NJCourtsHolidays()`. This function iterates through each year in the specified range and
    compiles all holiday dates and names returned by `NJCourtsHolidays()`

    Args:
        starting_year (int): First year to include (inclusive).
        ending_year (int): Last year to include (exclusive).

    Returns:
        pandas.DataFrame: A DataFrame with the following columns:
            - holiday_year (str): The year of the holiday.
            - holiday_date (str): Holiday date formatted as MM/DD/YYYY.
            - holiday_description (str): Name of the holiday.
    Example:
        >>>df_holidays = generate_holidays(starting_year=2025, ending_year=2040)
    """    
    data = {'holiday_year': [],
            'holiday_date': [],
            'holiday_description':[]}
    df = pd.DataFrame(data)
    
    for i in range(starting_year, ending_year):
        nj_holidays = NJCourtsHolidays(years=[i])
        for date_str, name in sorted(nj_holidays.items()):
            year = date_str.strftime('%Y')
            new_date_format = date_str.strftime('%m/%d/%Y')
            df.loc[len(df)] = [year, new_date_format, name]
    return df

In [84]:
#Example of Holidays in 2025
nj_holidays = NJCourtsHolidays(years=[2025])
for dates2, name in sorted(nj_holidays.items()):
     print(dates2, name)

2025-01-01 New Year's Day
2025-01-20 Martin Luther King Jr. Day
2025-02-17 Presidents Day
2025-04-18 Good Friday
2025-05-26 Memorial Day
2025-06-19 Juneteenth National Independence Day
2025-07-04 Independence Day
2025-09-01 Labor Day
2025-10-13 Columbus Day
2025-11-04 Election Day
2025-11-11 Veterans Day
2025-11-27 Thanksgiving Day
2025-11-28 Court Recess (Day after Thanksgiving)
2025-12-25 Christmas Day
2025-12-26 Court Recess (Day after Christmas)


In [86]:
df_holidays = generate_holidays(starting_year=2025, ending_year=2040)

In [88]:
df_holidays

,holiday_year,holiday_date,holiday_description
0,2025,01/01/2025,New Year's Day
1,2025,01/20/2025,Martin Luther King Jr. Day
2,2025,02/17/2025,Presidents Day
3,2025,04/18/2025,Good Friday
4,2025,05/26/2025,Memorial Day
...,...,...,...
231,2039,11/11/2039,Veterans Day
232,2039,11/24/2039,Thanksgiving Day
233,2039,11/25/2039,Court Recess (Day after Thanksgiving)
234,2039,12/25/2039,Christmas Day


In [51]:
df_holidays.to_csv("holidays.csv")